In [ ]:
from uuid import uuid4
from vllm import SamplingParams
import json
import re


async def fallback_generate_node(state: QAState) -> dict:

    prompt = prompt1.format(
        til=state.til,
        level=state.level
    )

    sampling_params = SamplingParams(
        temperature=0.7,
        max_tokens=2048,
        stop_token_ids=[2],  # EOS token
    )

    request_id = str(uuid4())
    final_text = ""

    async for output in llm.generate(
        prompt=prompt,
        sampling_params=sampling_params,
        request_id=request_id
    ):
        final_text = output.outputs[0].text.strip()

    return {
        "answer_raw": final_text
    }

# async def parsing_node(state: QAState) -> dict:
#     try:
#         parsed = json.loads(state.answer_raw)
#         qa_list = [ContentState(**item) for item in parsed]
#         return {"content": qa_list}
#     except Exception as e:
#         print(f"JSON 파싱 에러: {e}")
#         return {"content": []}

async def parsing_node(state: QAState) -> dict:
    raw = state.answer_raw
    qa_list = []

    # 정규식으로 "question: ... answer: ..." 패턴 추출
    pattern = re.findall(r"question:\s*(.*?)\nanswer:\s*(.*?)(?=\nquestion:|\Z)", raw, re.DOTALL)

    for q, a in pattern:
        qa_list.append(ContentState(
            question=q.strip(),
            answer=a.strip()
        ))

    # fallback
    if not qa_list:
        qa_list = [ContentState(
            question="LLM 출력 파싱 실패",
            answer=raw.strip()
        )]

    return {"content": qa_list}


In [ ]:
    # results = qdrant.search(
    #     collection_name=collection,
    #     query_vector=query_vector,
    #     limit=3
    # )

    # if not results or results[0].score < 0.75:
    #     similarity_score = 0.0
    #     retrieved_texts = []
    # else:
    #     similarity_score = results[0].score
    #     retrieved_texts = [hit.payload["text"] for hit in results if "text" in hit.payload]

In [ ]:
prom = """
당신은 사용자의 기술 학습 기록을 바탕으로, 기술 면접 질문을 생성하는 AI입니다.

아래 정보를 참고하여,
[TIL 본문] {til}
[RAG 검색 결과] {text}
[선택한 난이도] {level}

※ level에 따라 질문 수준을 조절해서 면접 질문을 작성해주세요:
- level "1": 깊은 기술 이해와 실무 경험 기반 질문
- level "2": 개념적 이해를 묻는 질문
- level "3": 기본 개념을 묻는 질문

모든 질문과 답변은 반드시 **한국어**로 작성하세요.

"""